In [3]:

#3e. Reveja o ansatz:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


L = 1.0          # Comprimento da corda
c = 1.0          # Velocidade da onda
N = 100          # Número de pontos na discretização
T = 2*np.pi      # Período total da animação
n_frames = 100   # Número de frames

# Discretização do espaço e tempo
x = np.linspace(0, L, N+2)  # Pontos espaciais (incluindo bordas)
t = np.linspace(0, T, n_frames)  # Tempos para a animação
dx = x[1] - x[0]            # Espaçamento espacial

# Construir a matriz A (segunda derivada)
A = np.zeros((N, N))
for i in range(N):
    if i > 0:
        A[i, i-1] = 1
    A[i, i] = -2
    if i < N-1:
        A[i, i+1] = 1
A = A / (dx**2)

# Calcular autovalores e autovetores
autovalores, autovetores = np.linalg.eigh(A)

# Frequências angulares (ω = c√(-λ))
omega = c * np.sqrt(-autovalores)

# Preparar a figura
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Animação dos 4 Primeiros Modos de Vibração', fontsize=16)
axs = axs.ravel()  # Achatar o array de eixos

# Inicializar linhas para cada modo
lines = []
for n in range(4):
    # Configurar cada subplot
    axs[n].set_xlim(0, L)
    axs[n].set_ylim(-1.5, 1.5)
    axs[n].set_title(f'Modo {n+1} - ω = {omega[n]:.2f}')
    axs[n].set_xlabel('Posição x')
    axs[n].set_ylabel('Amplitude u(x,t)')
    axs[n].grid(True)

    # Inicializar linha (parte real da solução)
    line, = axs[n].plot([], [], 'b-', lw=2)
    lines.append(line)

# Função de inicialização
def init():
    for line in lines:
        line.set_data([], [])
    return lines

# Função de animação
def animate(i):
    current_time = t[i]

    for n in range(4):
        # Obter o modo espacial (autovetor)
        phi_n = np.zeros(N+2)
        phi_n[1:-1] = autovetores[:, n]

        # Reconstruir u(x,t) = Re[φ(x) * e^(iωt)] = φ(x) * cos(ωt)
        u = phi_n * np.cos(omega[n] * current_time)

        # Normalizar para melhor visualização
        u = u / np.max(np.abs(phi_n)) if np.max(np.abs(phi_n)) > 0 else u

        # Atualizar a linha
        lines[n].set_data(x, u)

    return lines

# Criar a animação
ani = FuncAnimation(fig, animate, frames=n_frames,
                    init_func=init, blit=True, interval=50)

plt.tight_layout()
plt.close()

# Mostrar a animação (no Jupyter Notebook)
HTML(ani.to_jshtml())
ani.save('ex5.1-animation.gif', dpi=90)

MovieWriter ffmpeg unavailable; using Pillow instead.
